# KubeFlow Pipeline

### Installl prerequisites

In [17]:
!python3 -m pip install kfp --upgrade --user -q

You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
EXPERIMENT_NAME = 'kf pipeline'        # Name of the experiment in the UI
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'    # Base image used for components in the pipeline

In [8]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

print('kfp:', kfp.__version__)

kfp: 0.4.0


In [9]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=BASE_IMAGE
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(f'{a} + {b} = {a + b}')
    return a + b

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  after removing the cwd from sys.path.


In [11]:
add_op = components.func_to_container_op(
    add,
    base_image=BASE_IMAGE,
)

In [12]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a: float =0,
   b: float =7
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

In [13]:
# Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}
# Launch a pipeline run given the pipeline function definition
kfp.Client().create_run_from_pipeline_func(calc_pipeline, arguments=arguments, 
                                           experiment_name=EXPERIMENT_NAME)
# The generated links below lead to the Experiment page and the pipeline run details page, respectively

RunPipelineResult(run_id=bae1e423-8b30-4a21-b878-1989fbe7ae52)

In [14]:
# Compile the pipeline
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [15]:
# Get or create an experiment
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [16]:
# Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

# Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

# The generated link below leads to the pipeline run information page.